### Using bedrock prompts and prompt templates


Create reusable prompt templates using the create_prompt API

✅ Invoke foundation models with structured input using the converse API

✅ Optimize prompt performance with the optimize_prompt API

✅ Version and iterate on prompts

✅ Delete prompts from Prompt Management

In [1]:
import json
import datetime
from IPython.display import display, JSON
import boto3

In [2]:
bedrock_agent = boto3.client("bedrock-agent")
bedrock_runtime = boto3.client("bedrock-runtime")
bedrock_agent_runtime = boto3.client("bedrock-agent-runtime")

model_id = "amazon.nova-micro-v1:0"

In [3]:
### using create_prompt API

prompt_name = f"job-description-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

template_text = '''
You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words
'''
response = bedrock_agent.create_prompt(
    name=prompt_name,
    description="Generates inclusive job descriptions from structured inputs",
    defaultVariant="v1",
    variants=[
        {
            "name": "v1",
            "modelId": model_id,
            "templateType": "TEXT",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "job_title"},
                        {"name": "responsibilities"},
                        {"name": "requirements"},
                        {"name": "location"},
                        {"name": "work_type"}
                    ],
                    "text": template_text
                }
            },
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 500,
                    "temperature": 0.7,
                    "topP": 0.9,
                    "stopSequences": []
                }
            }
        }
    ]
)
print("\n==================== Response Object ====================\n")
display(JSON(response))

print("\n==================== Prompt ARN ====================\n")
print(response['arn'])


==================== Response Object ====================



<IPython.core.display.JSON object>


==================== Prompt ARN ====================

arn:aws:bedrock:eu-west-2:035663780217:prompt/MVNHMPCX3O


In [4]:
response = bedrock_agent.create_prompt_version(
    description='Initial prompt for creating job description documents.',
    promptIdentifier="arn:aws:bedrock:eu-west-2:035663780217:prompt/MVNHMPCX3O"
)

print("\n==================== PROMPT VERSION ARN ====================\n")
print(response["arn"])


==================== PROMPT VERSION ARN ====================

arn:aws:bedrock:eu-west-2:035663780217:prompt/MVNHMPCX3O:1


### Invoke model using versioned prompt template

To invoke the model using a prompt created using Prompt Management you need to use the converse API as documented in the 
boto3 documentation
, not the direct invoke_model API.

✏️ The Converse API in Amazon Bedrock is a high-level API that lets you invoke a model using resources like:

Prompts (created and stored with Prompt Management)

Knowledge Bases

Agents

Tools

Converse provides a consistent interface that works with all models that support messages. This allows you to write code once and use it with different models. If a model has unique inference parameters, you can also pass those unique parameters to the model.

Rather than sending a raw prompt string every time (as you would with the InvokeModel API), the Converse API allows you to pass in a promptId and supply the input variables.

In [5]:
prompt_arn_with_version = "arn:aws:bedrock:eu-west-2:035663780217:prompt/MVNHMPCX3O:1"

response = bedrock_runtime.converse(
    modelId=prompt_arn_with_version,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\n==================== Response Text ====================\n")
print(response['output']['message']['content'][0]['text'])


==================== Response Text ====================

**Job Title: UX Designer**

**Summary:**
We are seeking a talented and inclusive UX Designer to join our dynamic team. As a UX Designer, you will be responsible for designing intuitive user interfaces, conducting usability testing, and collaborating closely with our product teams to enhance user experience. This role offers flexibility with options to work in New York or remotely.

**Responsibilities:**
- Design compelling, user-friendly interfaces that meet user needs and business goals.
- Conduct usability testing to gather insights and refine designs.
- Collaborate with cross-functional teams to ensure seamless product development.
- Utilize tools like Figma, HTML, and CSS to bring designs to life.

**Requirements:**
- 3+ years of experience in UX design.
- Proficiency in Figma, HTML, and CSS.
- Strong communication skills to effectively convey ideas and collaborate with team members.
- Ability to work in a fast-paced, inclus

### Optimize the prompt

Bedrock has a tool to optimize prompts via the agent-runtime-service or management console

Optimization rewrites prompts to yield inference results that are more suitable for your use case. You can choose the model that you want to optimize the prompt for and then generate a revised prompt.

After you submit a prompt to optimize, Amazon Bedrock analyzes the components of the prompt. If the analysis is successful, it then rewrites the prompt. You can then copy and use the text of the optimized prompt and create a new variant for the prompt.

In [8]:
def handle_response_stream(response):
    try:
        event_stream = response['optimizedPrompt']
        for event in event_stream:
            if 'optimizedPromptEvent' in event:
                print("\n==================== OPTIMIZED PROMPT ====================\n")
                print(event['optimizedPromptEvent']['optimizedPrompt']['textPrompt']['text'])
    except Exception as e:
        raise e


prompt_input = {
    "textPrompt": {
        "text": template_text
    }
}

response = bedrock_agent_runtime.optimize_prompt(
            input=prompt_input,
            targetModelId=model_id
        )

# display(JSON(response))

print("\n==================== ORIGINAL PROMPT ====================\n")
print(template_text)
handle_response_stream(response)


==================== ORIGINAL PROMPT ====================


You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words


==================== OPTIMIZED PROMPT ====================

"# HR Assistant: Job Description Generator\n\n## Task\nCreate a professional, inclusive job description using the provided information.\n\n## Input Information\n### Job Title\n{{job_title}}\n\n### Responsibilities\n{{responsibilities}}\n\n### Requirements\n{{requirements}}\n\n### Location\n{{location}}\n\n### Work Type\n{{work_type}}\n\n## Output Guidelines\n1. Begin with a clear, compelling summary of the position (1-2 sentences)\n2. Structure the job description with these clearly labeled sections:\n   - About the Role\n   - Key Resp

In [9]:
prompt_identifier = "arn:aws:bedrock:eu-west-2:035663780217:prompt/MVNHMPCX3O:1"

existing_prompt = bedrock_agent.get_prompt(promptIdentifier=prompt_identifier)
print("\n==================== ORIGINAL PROMPT ====================\n")
print(existing_prompt["variants"][0]["templateConfiguration"]["text"]["text"])


updated_variants = existing_prompt["variants"]
for variant in updated_variants:
    if variant["templateType"] == "TEXT":
        variant["templateConfiguration"]["text"]["text"] = "# HR Assistant: Job Description Generator\n\n## Task\nCreate a professional, inclusive job description using the provided information.\n\n## Input Information\n### Job Title\n{{job_title}}\n\n### Responsibilities\n{{responsibilities}}\n\n### Requirements\n{{requirements}}\n\n### Location\n{{location}}\n\n### Work Type\n{{work_type}}\n\n## Output Guidelines\n1. Begin with a clear, compelling summary of the position (1-2 sentences)\n2. Structure the job description with these clearly labeled sections:\n   - About the Role\n   - Key Responsibilities\n   - Qualifications & Skills\n   - Location & Work Arrangement\n\n3. Format requirements:\n   - Use bullet points for responsibilities and requirements\n   - Employ active, gender-neutral language throughout\n   - Avoid unnecessary jargon or buzzwords\n   - Highlight essential qualifications separately from preferred ones\n   - Include a brief statement about company commitment to diversity and inclusion\n\n4. Keep the entire description under 250 words\n5. Use a professional, welcoming tone that appeals to diverse candidates\n\nGenerate the job description immediately without any preamble or additional explanation."


response = bedrock_agent.update_prompt(
    promptIdentifier=prompt_identifier,
    name=existing_prompt["name"],
    description=existing_prompt["description"],
    defaultVariant=existing_prompt["defaultVariant"],
    variants=updated_variants
)

print("\n==================== UPDATED PROMPT ====================\n")

updated_prompt = bedrock_agent.get_prompt(promptIdentifier=prompt_identifier)
print(updated_prompt["variants"][0]["templateConfiguration"]["text"]["text"])


==================== ORIGINAL PROMPT ====================


You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words


==================== UPDATED PROMPT ====================


You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words



In [10]:
response = bedrock_agent.create_prompt_version(
    description='Optimized prompt for creating job description documents.',
    promptIdentifier=prompt_identifier
)

print("\n==================== PROMPT VERSION ARN ====================\n")
print(response["arn"])


==================== PROMPT VERSION ARN ====================

arn:aws:bedrock:eu-west-2:035663780217:prompt/MVNHMPCX3O:2


In [11]:
prompt_arn = "arn:aws:bedrock:eu-west-2:035663780217:prompt/MVNHMPCX3O:1"

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

\n==================== Response Text ====================\n
**Job Title: UX Designer**

**Summary:**
We are seeking a talented and inclusive UX Designer to join our dynamic team in New York or remotely. As a UX Designer, you will play a pivotal role in creating intuitive and engaging user interfaces, conducting usability testing, and collaborating closely with our product teams to deliver exceptional user experiences.

**Responsibilities:**
- Design compelling and user-friendly interfaces that enhance the overall user experience.
- Conduct thorough usability testing to identify and resolve user pain points.
- Collaborate with cross-functional teams to ensure seamless integration of design into product development.
- Utilize Figma for prototyping and design iterations.
- Apply HTML/CSS knowledge to bring designs to life.

**Requirements:**
- 3+ years of experience in UX design.
- Proficiency in Figma, HTML, and CSS.
- Strong communication skills to effectively convey ideas and collabora

In [12]:
prompt_arn = "arn:aws:bedrock:eu-west-2:035663780217:prompt/MVNHMPCX3O:2"

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

\n==================== Response Text ====================\n
**UX Designer**

**About the Role**
Join our innovative team as a UX Designer to craft exceptional user experiences. We are looking for a creative professional to design intuitive user interfaces, conduct usability testing, and collaborate closely with product teams to enhance our digital products.

**Key Responsibilities**
- Design user interfaces that are intuitive and user-friendly
- Conduct and analyze usability testing to refine user experiences
- Collaborate with cross-functional teams to implement design solutions

**Qualifications & Skills**
- Essential: 
  - 3+ years experience in UX design
  - Proficiency in Figma
  - Knowledge of HTML/CSS
  - Strong communication skills
- Preferred:
  - Experience with user research and analytics
  - Familiarity with design systems and component libraries

**Location & Work Arrangement**
- Based in New York or fully remote

**Company Commitment**
We are dedicated to fostering a dive

In [13]:
response = bedrock_agent.list_prompts()

for prompt in response['promptSummaries']:
    prompt_id = prompt['id']  # The correct key is 'id', not 'promptId'
    print(f"Deleting prompt: {prompt['name']} (ID: {prompt_id})")
    bedrock_agent.delete_prompt(promptIdentifier=prompt_id)

print("All prompts deleted successfully")

Deleting prompt: job-description-20260115113631 (ID: MVNHMPCX3O)
All prompts deleted successfully
